<div style="background-color: darkslategray; color: white; padding: 15px; border-radius: 8px;">
    <center><h1 style="font-family: Arial, sans-serif;">Predicting Rare Species</h1></center>
    <center><h3 style="font-family: Arial, sans-serif;">Deep Learning Project</h3></center>
</div>

**<h3>Table of Contents</h3>**
* [1. Environment Setup](#1-environment-setup)
    * [1.1 Import Libraries](#11-import-libraries)
    * [1.2 Import Dataset](#12-import-dataset)
* [2. Exploratory Data Analysis](#2-exploratory-data-analysis)
    * [2.1 Visualizations](#21-visualizations)
    * [2.2 Import Dataset](#22-feature-engineering)
* [3. Splitting the Dataset](#3-splitting-the-dataset)
* [4. Splitting the Dataset](#4-preprocessing)
    * [4.1 Image Generator and Augmentation](#41-image-generator-and-augmentation)


<div class="alert alert-block alert-success">

# **1.** Environment Setup

<div>

## 1.1 Connect Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import os

# # Change to the directory where project is located
# os.chdir('/content/drive/MyDrive/College/MSc/2nd Semester/Deep Learning/project')

# # Verify that we changed the directory
# print("Changed directory to:", os.getcwd())

## 1.2 Import Libraries

In [1]:
# # # Google Colab
# !pip install keras_cv

In [2]:
pip install ipywidgets --upgrade

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot

Jupyter command `jupyter-nbextension` not found.


In [4]:
!pip install jupyterlab_widgets

In [18]:
import pandas as pd
import zipfile
import pickle
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import regularizers
from classes import *
from functions import *

In [19]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Dropout, Input, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.metrics import AUC, F1Score, CategoricalAccuracy, TopKCategoricalAccuracy
from sklearn.metrics import f1_score, precision_score, recall_score
from keras import optimizers

## 1.3 Import Dataset

The amount of data we have is not supported by GitHub (where we have our project stored). The solution is: create a folder named data and allocate the rare_species file inside it. The gitignore file makes sure this folder is not used when we are pulling or pushing changes but everyone needs to have it on their machines locally. A random seed was used to ensure that the splits stay the same. 

In [7]:
# # Google Colab

# # Define the path to the zip file and the extraction folder
# zip_path = Path("../data/rare_species.zip")

# # Get the directory where the zip file is located
# extract_path = Path("../data/rare_species")

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

In [8]:
# Import Metadata
metadata_path = Path("../data/rare_species/metadata.csv")
df = pd.read_csv(metadata_path)
df.head()

,rare_species_id,eol_content_id,eol_page_id,kingdom,phylum,family,file_path
0,75fd91cb-2881-41cd-88e6-de451e8b60e2,12853737,449393,animalia,mollusca,unionidae,mollusca_unionidae/12853737_449393_eol-full-si...
1,28c508bc-63ff-4e60-9c8f-1934367e1528,20969394,793083,animalia,chordata,geoemydidae,chordata_geoemydidae/20969394_793083_eol-full-...
2,00372441-588c-4af8-9665-29bee20822c0,28895411,319982,animalia,chordata,cryptobranchidae,chordata_cryptobranchidae/28895411_319982_eol-...
3,29cc6040-6af2-49ee-86ec-ab7d89793828,29658536,45510188,animalia,chordata,turdidae,chordata_turdidae/29658536_45510188_eol-full-s...
4,94004bff-3a33-4758-8125-bf72e6e57eab,21252576,7250886,animalia,chordata,indriidae,chordata_indriidae/21252576_7250886_eol-full-s...


In [9]:
df.shape # 11983 images

(11983, 7)

<div class="alert alert-block alert-success">

# **2.** Exploratory Data Analysis !!!Not Needed

<div>

In [ ]:
# kingdom_counts = df['kingdom'].value_counts()
# plot_graph(title='Distribution of Observations Across Kingdom', xlabel='Kingdom', ylabel='Number of Observations',  counts=kingdom_counts)

In [ ]:
# phylum_counts = df['phylum'].value_counts()
# plot_graph(title='Distribution of Observations Across Phylum', xlabel='Phylum', ylabel='Number of Observations',  counts=phylum_counts)

In [ ]:
# family_counts = df['family'].value_counts()
# plot_graph(title='Distribution of Observations Across Family', xlabel='Family', ylabel='Number of Observations',  counts=family_counts)

In [ ]:
# family_counts = df["family"].value_counts(normalize=True)*100

# df_family_proportions = pd.DataFrame({
#     "Family": family_counts.index,
#     "Proportion (%)": family_counts.values  
# })

# df_family_proportions # 202 different families

In [ ]:
# num_families_025 = int((df_family_proportions["Proportion (%)"] <= 0.2505).sum())
# num_families_025 # 118 families have less than 0.2505% of images

In [ ]:
# top_100_families = df['family'].value_counts().nlargest(100)
# plot_graph(title='Distribution of Observations Across Top 100 Families', xlabel='Family', ylabel='Number of Observations',  counts=top_100_families)

In [ ]:
# top_84_family_proportion_sum = df_family_proportions.iloc[:84]["Proportion (%)"].sum()
# top_84_family_proportion_sum # although 118 families represent less than 0.25% of the dataset, the 84 that have >0.25% represent only 70.46% of the dataset

<!-- is it better to predict phylum and then family or a concatenation of the two? -->

In [ ]:
# df["phylum_family"] = df["phylum"] + "_" + df["family"]
# df

In [ ]:
# family_phylum_counts = df.groupby("phylum")["family"].nunique()

In [ ]:
# family_phylum_counts # the phylum with the most families is chordata

In [ ]:
# anomaly_counts = df.groupby("family")["phylum"].nunique()

In [ ]:
# anomaly_counts.value_counts() # there are no families that belong to more than 1 phylum

In [ ]:
# phylum_family_counts = df["phylum_family"].value_counts(normalize=True)*100

# df_phylum_family_proportions = pd.DataFrame({
#     "phylum_family": phylum_family_counts.index,
#     "Proportion (%)": phylum_family_counts.values.round(4)  
# })

# df_phylum_family_proportions # 202 different families

<div class="alert alert-block alert-success">

# **3.** Feature Engineering !!!Not Needed

<div>

In [22]:
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
import os
import tensorflow as tf

# Encode family (target) and phylum (metadata feature)
family_encoder = LabelEncoder()
phylum_encoder = LabelEncoder()
df["family_encoded"] = family_encoder.fit_transform(df["family"])
df["phylum_encoded"] = phylum_encoder.fit_transform(df["phylum"])

# Check the number of unique families and phyla
num_families = df["family_encoded"].nunique()
num_phyla = df["phylum_encoded"].nunique()
print(f"Number of unique phyla: {num_phyla}")
print(f"Number of unique families: {num_families}")

# One-hot encode the family and phylum columns using NumPy (not eager tensors)
df["phylum_onehot"] = df["phylum_encoded"].apply(lambda x: tf.one_hot(x, depth=num_phyla).numpy())
df["family_onehot"] = df["family_encoded"].apply(lambda x: tf.one_hot(x, depth=num_families).numpy())

# Create a new column for the full file path
# Make it OS-independent using Path
df["full_file_path"] = df["file_path"].apply(lambda x: str(Path("../data/rare_species") / x))


Number of unique phyla: 5
Number of unique families: 202


In [ ]:
# # Encode family (target) and phylum (metadata feature)
# family_encoder = LabelEncoder()
# phylum_encoder = LabelEncoder()
# df["family_encoded"] = family_encoder.fit_transform(df["family"])
# df["phylum_encoded"] = phylum_encoder.fit_transform(df["phylum"])

# # Check the number of unique families and phyla
# num_families = df["family_encoded"].nunique()
# num_phyla = df["phylum_encoded"].nunique()
# print(f"Number of unique phyla: {num_phyla}")
# print(f"Number of unique families: {num_families}")

# # One-hot encode the family and phylum columns
# df["phylum_onehot"] = df["phylum_encoded"].apply(lambda x: tf.one_hot(x, depth=num_phyla).numpy())
# df["family_onehot"] = df["family_encoded"].apply(lambda x: tf.one_hot(x, depth=num_families).numpy())

# # Create a new column for the full file path
# # Windows
# df["full_file_path"] = df["file_path"].apply(lambda x: os.path.join(path, x.replace("/", "\\")))
# # Mac
# df["full_file_path"] = df["file_path"].apply(lambda x: str(Path("../data/rare_species") / x))

# # Increase display width
# # pd.set_option('display.max_colwidth', None)
# # pd.reset_option('display.max_colwidth')
# df.head()

<div class="alert alert-block alert-success">

# **4.** Splitting the Dataset !!!Not Needed

<div>

In [23]:
# Split the data into training, validation, and testing sets
train_df, test_valid_df = train_test_split(df, test_size=0.3, shuffle=True, random_state=42, stratify=df['family'])
valid_df, test_df = train_test_split(test_valid_df, test_size=0.5, shuffle=True, random_state=42, stratify=test_valid_df['family'])

In [24]:
# # Organize the data into train/val/test directories
image_base_path = Path("../data/rare_species")
base_output_dir = image_base_path

# # Copy the images to the new directories
organize_split(image_base_path=image_base_path, base_output_dir=base_output_dir, split_df=train_df, split_name="train")
organize_split(image_base_path=image_base_path, base_output_dir=base_output_dir, split_df=valid_df, split_name="val")
organize_split(image_base_path=image_base_path, base_output_dir=base_output_dir, split_df=test_df, split_name="test")

#Clean up unnecessary folders
cleanup_folders(image_base_path)

In [25]:
train_df['full_file_path'][0]

'..\\data\\rare_species\\mollusca_unionidae\\12853737_449393_eol-full-size-copy.jpg'

In [26]:
# # Update the file paths to point to the new directories
train_df = update_paths(train_df, "train")
valid_df = update_paths(valid_df, "val")
test_df = update_paths(test_df, "test")

In [27]:
train_df['full_file_path'][1]

'..\\data\\rare_species\\train\\geoemydidae\\20969394_793083_eol-full-size-copy.jpg'

In [ ]:
# # Export each DataFrame as a .pkl file
#with open("../data/train_df.pkl", "wb") as f:
#   pickle.dump(train_df, f)

# with open("../data/valid_df.pkl", "wb") as f:
#    pickle.dump(valid_df, f)

#with open("../data/test_df.pkl", "wb") as f:
#    pickle.dump(test_df, f)

In [ ]:
# # sampling train
# train_df_sampled, _ = train_test_split(train_df, test_size=0.5, shuffle=True, random_state=42, stratify=train_df['family'])
# organize_split(image_base_path=image_base_path, base_output_dir=base_output_dir, split_df=train_df_sampled, split_name="train_sampled")
# train_df_sampled = update_paths(train_df_sampled, "train_sampled")

In [ ]:
# train_df_counts = train_df['family'].value_counts().nlargest(100)
# plot_graph(title='Distribution of Observations - Train Dataset', xlabel='Family', ylabel='Number of Observations',  counts=train_df_counts )

# valid_df_counts = valid_df['family'].value_counts().nlargest(100)
# plot_graph(title='Distribution of Observations - Validation Dataset', xlabel='Family', ylabel='Number of Observations',  counts=valid_df_counts )

# test_df_counts = test_df['family'].value_counts().nlargest(100)
# plot_graph(title='Distribution of Observations - Test Dataset', xlabel='Family', ylabel='Number of Observations',  counts=test_df_counts )

In [ ]:
# print(f"Training samples: {len(train_df)}")
# # print(f"Training sampled samples: {len(train_df_sampled)}")
# print(f"Validation samples: {len(valid_df)}")
# print(f"Testing samples: {len(test_df)}")

<div class="alert alert-block alert-success">

# **5.** Preprocessing

<div>

- Normalizes pixel values (e.g., rescaling from [0,255] to [0,1]).
- Resizes images to a fixed size (e.g., 224x224 pixels).
- Applies augmentation (only during training).
- Converts images to batches (e.g., batch_size=32 loads 32 images at a time).

In [ ]:
# Load the DataFrames from the .pkl files
with open("../data/rare_species/train_df.pkl", "rb") as f:
    train_df = pickle.load(f)

with open("../data/rare_species/valid_df.pkl", "rb") as f:
    valid_df = pickle.load(f)

with open("../data/rare_species/test_df.pkl", "rb") as f:
    test_df = pickle.load(f)

In [ ]:
# minority_class = train_df['family'].value_counts()[train_df['family'].value_counts() < 25].index
# minority_class=minority_class.to_list()

In [ ]:
# batch_size = 32 ## the less the better because in each epoch the model sees N / batch_size images
# image_size = (224, 224)

# preprocess = Preprocessor(image_size=image_size, batch_size=batch_size)

# train_ds, class_names = preprocess.load_img(data_dir="../data/rare_species/train", minority_class=minority_class, augment='mixup', oversampling=True)
# train_df_sampled, class_names = preprocess.load_img(data_dir="../data/rare_species/train_sampled", minority_class=minority_class, augment='mixup', oversampling=True)
# val_ds, _ = preprocess.load_img(data_dir="../data/rare_species/val", minority_class=minority_class, augment=None, oversampling=False)
# test_ds, _ = preprocess.load_img(data_dir="../data/rare_species/tes", minority_class=minority_class, augment=None, oversampling=False)

In [ ]:
# num_images = 16 ##
# rows, cols = 4, 4 ##

# plot_batch(train_ds, class_names=class_names, num_images=num_images, rows=rows, cols=cols)

### With Phylum

In [ ]:
# Create the datasets for training, validation, and testing with both image and phylum as inputs
# train_ds = build_ds_with_phylum(train_df)
# val_ds = build_ds_with_phylum(valid_df)
# test_ds = build_ds_with_phylum(test_df)


# train_ds

## 5.1 Outliers

In [ ]:
# import os
# from glob import glob

# train_dir = Path("../data/rare_species/train")

# image_paths = glob(os.path.join(train_dir, "*", "*.jpg"))  # ou .png dependendo do teu caso

# print(f"Total de imagens encontradas: {len(image_paths)}")

In [ ]:
# from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
# from tensorflow.keras.preprocessing import image
# from sklearn.cluster import DBSCAN
# import numpy as np
# import cv2

# # Modelo ResNet50 para extrair embeddings
# model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# def extract_embedding(img_path):
#     try:
#         img = image.load_img(img_path, target_size=(224, 224))
#         x = image.img_to_array(img)
#         x = preprocess_input(np.expand_dims(x, axis=0))
#         return model.predict(x, verbose=0)[0]
#     except Exception as e:
#         print(f"Erro com {img_path}: {e}")
#         return None

# # Extrair os embeddings
# embeddings = []
# valid_paths = []

# for path in image_paths:
#     emb = extract_embedding(path)
#     if emb is not None:
#         embeddings.append(emb)
#         valid_paths.append(path)

# embeddings = np.array(embeddings)


In [ ]:
# len(embeddings)

In [ ]:
# from sklearn.metrics.pairwise import pairwise_distances
# import matplotlib.pyplot as plt
# import numpy as np

# # Calcula a matriz de distâncias entre embeddings
# dists = pairwise_distances(embeddings)

# # Para cada ponto, encontra a distância ao seu 5º vizinho mais próximo
# sorted_dists = np.sort(dists, axis=1)
# k_distances = sorted_dists[:, 10]  # min_samples = 5

# # Plot
# plt.figure(figsize=(10, 5))
# plt.plot(np.sort(k_distances))
# plt.title("K-distance plot (5º vizinho)")
# plt.xlabel("Ponto")
# plt.ylabel("Distância ao 5º vizinho")
# plt.grid(True)
# plt.show()


In [ ]:
# from sklearn.cluster import DBSCAN
# import numpy as np

# # DBSCAN: ajusta eps conforme o teu dataset
# clustering = DBSCAN(eps=45, min_samples=10).fit(embeddings)
# labels = clustering.labels_

# # -1 indica outliers
# outliers = np.where(labels == -1)[0]
# outlier_paths = [valid_paths[i] for i in outliers]

# print(f"Número de outliers encontrados: {len(outlier_paths)}")

In [ ]:
# len(outlier_paths)

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

# def show_images(image_paths, n=20, cols=5, title="Outliers"):
#     plt.figure(figsize=(15, (n // cols + 1) * 3))
#     for i, img_path in enumerate(image_paths[:n]):
#         img = mpimg.imread(img_path)
#         plt.subplot(n // cols + 1, cols, i + 1)
#         plt.imshow(img)
#         plt.axis("off")
#         plt.title(os.path.basename(img_path), fontsize=8)
#     plt.suptitle(title, fontsize=16)
#     plt.tight_layout()
#     plt.show()

# show_images(outlier_paths, n=100, cols=6)


## 5.2 Augmentations

In [ ]:
# augmentations_to_test = [
#     # "none",
#     # "light",
#     # "medium",
#     # "heavy",
#     # "grayscale",
#     # "randaugment",
#     "mixup",
#     "cutmix"]

<!-- ##### Simple model do test augmentations -->

In [ ]:
# def build_model(num_classes):
#     base = keras.applications.EfficientNetB0(
#         input_shape=(224, 224, 3),
#         include_top=False,
#         weights="imagenet",
#         pooling="avg"
#     )
#     base.trainable = False  # You can fine-tune later

#     inputs = keras.Input(shape=(224, 224, 3))
#     x = base(inputs, training=False)
#     x = keras.layers.Dropout(0.2)(x)
#     outputs = keras.layers.Dense(num_classes, activation="softmax")(x)

#     return keras.Model(inputs, outputs)

In [ ]:
# def sample_dataset(dataset, fraction=None, num_batches=None, seed=42):
#     """Return a sampled subset of the dataset."""
#     if fraction:
#         dataset = dataset.shuffle(1000, seed=seed)
#         dataset = dataset.take(int(fraction * tf.data.experimental.cardinality(dataset).numpy()))
#     elif num_batches:
#         dataset = dataset.take(num_batches)
#     return dataset

<!-- ##### Loop -->

In [ ]:
# # Initialize the preprocessor
# pre = Preprocessor(image_size=(224, 224), batch_size=32)

# # Store results
# results = {}

# # Loop through each augmentation
# for aug in augmentations_to_test:
#     print(f"\nTraining with augmentation: {aug}")

#     # Load datasets
#     train_ds, class_names = pre.load_img(
#         data_dir="../data/rare_species/train",
#         augment=aug
#     )

#     val_ds, _ = pre.load_img(
#         data_dir="../data/rare_species/val",
#         augment=None
#     )

#     # Sample a subset of training data
#     train_ds = sample_dataset(train_ds, fraction=0.5)

#     # Build a fresh model (you should define this function)
#     model = build_sequential_model(list_of_layers=layers)

#     # Compile
#     model.compile(
#         optimizer="adam",
#         loss="categorical_crossentropy",
#         metrics=["accuracy"]
#     )

#     # Train
#     history = model.fit(
#         train_ds,
#         validation_data=val_ds,
#         epochs=5,
#         verbose=1
#     )

#     # Predict entire validation set at once
#     preds = model.predict(val_ds)
#     y_pred = np.argmax(preds, axis=1)

#     # Extract true labels in order
#     y_true = np.concatenate([np.argmax(y.numpy(), axis=1) for _, y in val_ds])

#     # Compute metrics
#     f1_macro = f1_score(y_true, y_pred, average='macro')
#     f1_weighted = f1_score(y_true, y_pred, average='weighted')
#     precision = precision_score(y_true, y_pred, average='weighted')
#     recall = recall_score(y_true, y_pred, average='weighted')

#     # Store in results
#     results[aug] = {
#         "val_accuracy": history.history["val_accuracy"][-1],
#         "f1_macro": f1_macro,
#         "f1_weighted": f1_weighted,
#         "precision": precision,
#         "recall": recall
#     }

#     print(f"Finished '{aug}'")
#     print(f"  Accuracy:      {results[aug]['val_accuracy']:.4f}")
#     print(f"  F1 (macro):    {results[aug]['f1_macro']:.4f}")
#     print(f"  F1 (weighted): {results[aug]['f1_weighted']:.4f}")
#     print(f"  Precision:     {results[aug]['precision']:.4f}")
#     print(f"  Recall:        {results[aug]['recall']:.4f}")


In [ ]:
# augmentations_to_test = [
#     "none",
#     "light",
#     "medium",
#     "heavy",
#     "grayscale",
#     "randaugment",
#     "mixup",
#     "cutmix"]

In [ ]:
# # Initialize the preprocessor
# pre = Preprocessor(image_size=(224, 224), batch_size=32)

# # Store results
# results = {}

# # Loop through each augmentation
# for aug in augmentations_to_test:
#     print(f"\nTraining with augmentation: {aug}")

#     # Load datasets
#     train_ds, class_names = pre.load_img(
#         data_dir="../data/rare_species/train",
#         augment=aug
#     )

#     val_ds, _ = pre.load_img(
#         data_dir="../data/rare_species/val",
#         augment=None
#     )

#     # Sample a subset of training data
#     train_ds = sample_dataset(train_ds, fraction=0.5)

#     # Build a fresh model (you should define this function)
#     model = build_sequential_model(list_of_layers=layers)

#     # Compile
#     model.compile(
#         optimizer="adam",
#         loss="categorical_crossentropy",
#         metrics=["accuracy"]
#     )

#     # Train
#     history = model.fit(
#         train_ds,
#         validation_data=val_ds,
#         epochs=15,
#         verbose=1
#     )

#     # Predict entire validation set at once
#     preds = model.predict(val_ds)
#     y_pred = np.argmax(preds, axis=1)

#     # Extract true labels in order
#     y_true = np.concatenate([np.argmax(y.numpy(), axis=1) for _, y in val_ds])

#     # Compute metrics
#     f1_macro = f1_score(y_true, y_pred, average='macro')
#     f1_weighted = f1_score(y_true, y_pred, average='weighted')
#     precision = precision_score(y_true, y_pred, average='weighted')
#     recall = recall_score(y_true, y_pred, average='weighted')

#     # Store in results
#     results[aug] = {
#         "val_accuracy": history.history["val_accuracy"][-1],
#         "f1_macro": f1_macro,
#         "f1_weighted": f1_weighted,
#         "precision": precision,
#         "recall": recall
#     }

#     print(f"Finished '{aug}'")
#     print(f"  Accuracy:      {results[aug]['val_accuracy']:.4f}")
#     print(f"  F1 (macro):    {results[aug]['f1_macro']:.4f}")
#     print(f"  F1 (weighted): {results[aug]['f1_weighted']:.4f}")
#     print(f"  Precision:     {results[aug]['precision']:.4f}")
#     print(f"  Recall:        {results[aug]['recall']:.4f}")


<div class="alert alert-block alert-success">

# **6.** Models

<div>

## Custom - Carolina

In [11]:
minority_class = train_df['family'].value_counts()[train_df['family'].value_counts() < 25].index
minority_class=minority_class.to_list()

In [25]:
len(minority_class)

118

In [12]:
train_df['full_file_path'][0]

'../data/rare_species/train/unionidae/12853737_449393_eol-full-size-copy.jpg'

In [13]:
from PIL import Image

path = '../data/rare_species/train/unionidae/12853737_449393_eol-full-size-copy.jpg'

try:
    img = Image.open(path)
    img.verify()  # Check for corruption
    print("Image is valid!")
except Exception as e:
    print("Error:", e)

Image is valid!


In [14]:
train_ds, steps_per_epoch = build_ds_with_phylum_augmentation(train_df,minority_class=minority_class, oversampling=True, train=True)
val_ds, _ = build_ds_with_phylum_augmentation(valid_df)
test_ds, _ = build_ds_with_phylum_augmentation(test_df)

In [15]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Concatenate

# Inputs
num_families=202
image_input = Input(shape=(224, 224, 3), name="image_input")
phylum_input = Input(shape=(5,), name="phylum_input")

# Image branch (a small CNN from scratch)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(image_input)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = GlobalAveragePooling2D()(x)

# Concatenate with phylum input
x = Concatenate()([x, phylum_input])

# Dense layers
x = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
x = Dropout(0.2)(x)

# Output layer for family classification
output = Dense(num_families, activation='softmax')(x)  # set num_families accordingly

# Create model
model = Model(inputs=[image_input, phylum_input], outputs=output)


In [16]:
metrics = [
    CategoricalAccuracy(name="accuracy"),
    AUC(name="auc"),
    F1Score(average="macro", name="f1_macro"),
    F1Score(average="weighted", name="f1_weighted"),
    TopKCategoricalAccuracy(k=5, name="top5_accuracy")
]


loss_fn = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)

# Compile the model
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=metrics
)

In [17]:
# Initialize the experiment
experiment = Experiment(
    model=model,
    train_ds=train_ds,
    val_ds=val_ds,
    experiment_name="functional_model_phylum_grey_rotation",
    batch_size=32,
    image_size=(224, 224)
)

# Default callbacks
# ExperimentLogger,  # Log experiment results after each epoch
# EarlyStopping(patience=3, restore_best_weights=True),  # Early stopping callback
# ModelCheckpoint(checkpoint_file, save_best_only=True)  # Save best model based on validation performance

# Add callbacks
callbacks =[
    ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=4,
        min_lr=1e-6,
        verbose=1
    )
]
n_epochs=20
# Run the experiment
history = experiment.run_experiment(callbacks=callbacks, epochs=n_epochs, steps_per_epoch=steps_per_epoch) # callbacks=None use default callbacks (created in the Experiment class)

No checkpoint found, starting from scratch.
Epoch 1/20


KeyboardInterrupt: 

In [ ]:
next(iter(train_ds))

## With Phylum (Marco)

In [ ]:
# Image input
image_input = Input(shape=(224, 224, 3), name="image_input")
base_model = ResNet50(include_top=False, weights="imagenet", input_tensor=image_input)
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Phylum input
phylum_input = Input(shape=(5,), name="phylum_input")

# Combine both
combined = Concatenate()([x, phylum_input])
combined = Dense(256, activation='relu')(combined)
combined = Dropout(0.3)(combined)
output = Dense(202, activation='softmax')(combined)

model = Model(inputs=[image_input, phylum_input], outputs=output)

# Define the metrics
metrics = [
    CategoricalAccuracy(name="accuracy"),
    AUC(name="auc"),
    F1Score(average="macro", name="f1_macro"),
    F1Score(average="weighted", name="f1_weighted"),
    TopKCategoricalAccuracy(k=5, name="top5_accuracy")
]

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=metrics
)


TopKCategoricalAccuracy(k=5): https://www.tensorflow.org/api_docs/python/tf/keras/metrics/TopKCategoricalAccuracy

This is a more forgiving metric. It considers a prediction correct if the true label is among the model’s top 5 predictions (sorted by probability).
Useful when there are many classes (like 202 families) — even if top-1 is hard, top-5 can still show useful trends

In [ ]:
# Initialize the experiment
experiment = Experiment(
    model=model,
    train_ds=train_ds,
    val_ds=val_ds,
    experiment_name="resnet50_with_phylum",
    batch_size=32,
    image_size=(224, 224)
)

# Default callbacks
# ExperimentLogger,  # Log experiment results after each epoch
# EarlyStopping(patience=3, restore_best_weights=True),  # Early stopping callback
# ModelCheckpoint(checkpoint_file, save_best_only=True)  # Save best model based on validation performance

# Add callbacks
# callbacks = [

# ]

# Run the experiment
history = experiment.run_experiment(callbacks=None, epochs=10) # callbacks=None use default callbacks (created in the Experiment class)

## Sequential

In [ ]:
layers = [Conv2D(16, (3, 3), activation="relu", input_shape=(224, 224, 3)),
          MaxPooling2D((2, 2)),
          Conv2D(32, (3, 3), activation="relu"),
          MaxPooling2D((2, 2)),
          Conv2D(64, (3, 3), activation="relu"),
          MaxPooling2D((2, 2)),
          Flatten(),
          Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
          Dense(202, activation="softmax")
          ]

In [ ]:
first_sequential_model = build_sequential_model(list_of_layers=layers)
first_sequential_model.summary()

In [ ]:
n_epochs = 1 ##
batch_size = 32 ##
initial_lr = 0.01 ##
final_lr = 0.001 ##

verbose = 1
metrics = [
    CategoricalAccuracy(name="accuracy"),
    AUC(name="auc"),
    F1Score(average="macro", name="f1_score")
]

In [ ]:
my_scheduler_fn = lr_scheduler(initial_lr, final_lr, n_epochs)
callbacks = get_callbacks(
    checkpoint_file_path="checkpoints/model.keras",
    metrics_file_path="logs/run1.csv",
    lr_scheduler=my_scheduler_fn
)

In [ ]:

first_sequential_model.compile(optimizer='rmsprop', loss= 'categorical_crossentropy', metrics= metrics)
history = first_sequential_model.fit(train_ds, epochs= n_epochs, verbose= 1, batch_size = batch_size, validation_data= val_ds, callbacks=callbacks)

In [ ]:
acc_train = history.history['accuracy']
acc_val = history.history['val_accuracy']
plot_model_acc(num_epochs=n_epochs, train_acc=acc_train, val_acc=acc_val)

In [ ]:
loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
plot_model_loss(num_epochs=n_epochs, train_loss=loss_train, val_loss=loss_val)

In [ ]:
#Evaluate the Model on Test Data
test_loss, test_accuracy = model.evaluate(test_generator)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

## Non Sequential

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D


In [ ]:
verbose = 1
metrics = [
    CategoricalAccuracy(name="acc"),
    AUC(name="auc"),
    F1Score(average="macro", name="f1_score")
]

initial_lr = 1e-3  
final_lr = 1e-4
n_epochs = 50
my_scheduler_fn = lr_scheduler(initial_lr, final_lr, n_epochs)


from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Junta ao teu callbacks list
callbacks = get_callbacks(
    checkpoint_file_path="checkpoints/model.keras",
    metrics_file_path="logs/run2.csv",
    lr_scheduler=my_scheduler_fn
) + [early_stopping]

In [ ]:
input_img = Input(shape=(224, 224, 3))

x = Conv2D(16, (3, 3), activation="relu", padding='same')(input_img)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(32, (3, 3), activation="relu", padding='same')(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), activation="relu", padding='same')(x)
x = MaxPooling2D((2, 2))(x)

x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(202, activation='softmax')(x)

model = models.Model(inputs=input_img, outputs=output)

In [ ]:
model.compile(optimizer=optimizers.RMSprop(learning_rate=1e-3), loss='categorical_crossentropy', metrics=metrics)
history = model.fit(train_ds, epochs=20, batch_size=20, validation_data=val_ds, callbacks=callbacks)

## ResNet50 - Margarida

In [ ]:
verbose = 1
metrics = [
    CategoricalAccuracy(name="acc"),
    AUC(name="auc"),
    F1Score(average="macro", name="f1_score")
]

initial_lr = 1e-4  
final_lr = 1e-5
n_epochs = 50
my_scheduler_fn = lr_scheduler(initial_lr, final_lr, n_epochs)


callbacks = get_callbacks(
    checkpoint_file_path="checkpoints/model.keras",
    metrics_file_path="logs/run2.csv",
  lr_scheduler=my_scheduler_fn
)

In [ ]:
input_img = Input(shape=(224, 224, 3))

base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_img)

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(202, activation='softmax')(x)

# Define final model
model = models.Model(inputs=input_img, outputs=output)

In [ ]:
base_model.trainable = False
model.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='categorical_crossentropy', metrics=metrics)
history = model.fit(train_ds, epochs=100, batch_size=16, validation_data=val_ds, callbacks=callbacks)

In [ ]:
loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
plot_model_acc(num_epochs=5, train_loss=loss_train, train_val=loss_val)

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
xaxis = range(1,epochs+1)
plt.plot(xaxis, loss_train, 'g', label='Training loss')
plt.plot(xaxis, loss_val, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#Evaluate the Model on Test Data
test_loss, test_accuracy = model.evaluate(test_generator)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

## ResNet50 - Margarida - com preprocessing

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
# Load datasets
train_ds, class_names = preprocess.load_img(
    data_dir="../data/rare_species/train",
    minority_class=minority_class,
    augment='mixup',
    oversampling=True,
    preprocessing_function=preprocess_input
)

val_ds, _ = preprocess.load_img(
    data_dir="../data/rare_species/val",
    minority_class=minority_class,
    augment=None,
    preprocessing_function=preprocess_input
)

In [ ]:
verbose = 1
metrics = [
    CategoricalAccuracy(name="accuracy"),
    AUC(name="auc"),
    F1Score(average="macro", name="f1_score")
]

initial_lr = 1e-4  
final_lr = 1e-5
n_epochs = 50
my_scheduler_fn = lr_scheduler(initial_lr, final_lr, n_epochs)


In [ ]:
input_img = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_img)

for layer in base_model.layers:
    layer.trainable = False

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(202, activation='softmax', kernel_regularizer=regularizers.l2(1e-4))(x)

model = models.Model(inputs=base_model.input, outputs=x)

model.compile(
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=metrics
)


In [ ]:
experiment = Experiment(
    model=model,
    train_ds=train_ds,
    val_ds=val_ds,
    experiment_name="resnet50_with_preprocessing_pre_finetuning",
    batch_size=32,
    image_size=(224, 224)
)

# Callbacks to reduce overfitting
callbacks_additional = [
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,           # Reduce LR if no improvement after 2 epochs
        min_lr=1e-6,
        verbose=1
    )]
    
# Run the experiment
history = experiment.run_experiment(callbacks=callbacks_additional, epochs=7) # callbacks=None use default callbacks (created in the Experiment class)

In [ ]:
history = model.fit(train_ds, epochs=7, batch_size=32, validation_data=val_ds, callbacks=callbacks)

In [ ]:
set_trainable = False
for layer in base_model.layers:
    if layer.name == "conv5_block1_1_conv":
        set_trainable = True
    layer.trainable = set_trainable

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=metrics)

history = model.fit(train_ds, epochs=50, batch_size=32, validation_data=val_ds, callbacks=callbacks)

In [ ]:

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 1
print("Fitting the end-to-end model")
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

## EfficientNet

In [ ]:
verbose = 1
metrics = [
    CategoricalAccuracy(name="acc"),
    AUC(name="auc"),
    F1Score(average="macro", name="f1_score")
]

#initial_lr = 1e-4  
#final_lr = 1e-5
n_epochs = 50
#my_scheduler_fn = lr_scheduler(initial_lr, final_lr, n_epochs)


callbacks = get_callbacks(
    checkpoint_file_path="checkpoints/model.keras",
    metrics_file_path="logs/run2.csv",
#   lr_scheduler=my_scheduler_fn
)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model

# Número de classes (families) — substitui com o teu número real
num_classes = len(df['family'].unique())

base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = True

# congela só as primeiras camadas
for layer in base_model.layers[:150]:
    layer.trainable = False

# Topo personalizado
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(202, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=output)

# Compilar
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=metrics)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint("checkpoints/model.keras", monitor='val_loss', save_best_only=True),
    CSVLogger("logs/run2.csv")
]

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=callbacks
)